In [1]:
using BenchmarkTools

In [2]:
function MaxPool_v1(input, pool_size, gradient)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    gradient_height, gradient_width = size(gradient)
    
    input_height % pool_height != 0 ? input_height_new = pool_height*size(gradient)[1] : input_height_new = input_height
    input_width % pool_width != 0 ? input_width_new = pool_width*size(gradient)[2] : input_width_new = input_width
    
    J = zeros(Float32, input_height, input_width, channels)
        
    for c in 1:channels
        for i in 1:pool_width:input_width_new
            for j in 1:pool_height:input_height_new
                end_i = min(i + pool_width - 1, input_width)
                end_j = min(j + pool_height - 1, input_height)
            
                max_value, max_idx = findmax(input[i:end_i, j:end_j,c])
                    
                J[i + max_idx[1] - 1, j + max_idx[2] - 1,c] = 1*gradient[div(i-1,pool_width) + 1, div(j-1, pool_height) + 1, c]
            end
        end
    end

    return tuple(J)
end


MaxPool_v1 (generic function with 1 method)

In [3]:
function MaxPool_v2(input, pool_size, gradient)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    gradient_height, gradient_width = size(gradient)
    
    input_height % pool_height != 0 ? input_height_new = pool_height*size(gradient)[1] : input_height_new = input_height
    input_width % pool_width != 0 ? input_width_new = pool_width*size(gradient)[2] : input_width_new = input_width
    
    J = zeros(Float32, input_height, input_width, channels)
        
    for c in 1:channels
        for j in 1:pool_height:input_height_new
            for i in 1:pool_width:input_width_new
                end_i = min(i + pool_width - 1, input_width)
                end_j = min(j + pool_height - 1, input_height)
            
                max_value, max_idx = findmax(@views input[i:end_i, j:end_j,c])
                    
                J[i + max_idx[1] - 1, j + max_idx[2] - 1,c] = 1*gradient[div(i-1,pool_width) + 1, div(j-1, pool_height) + 1, c]
            end
        end
    end

    return tuple(J)
end

MaxPool_v2 (generic function with 1 method)

In [4]:
input = rand(Float32, 28, 28, 100);
gradient= rand(Float32, 14, 14, 100);

In [5]:
@benchmark MaxPool_v1(input, (2,2), gradient)

BenchmarkTools.Trial: 5632 samples with 1 evaluation.
 Range (min … max):  828.083 μs …   3.835 ms  ┊ GC (min … max): 0.00% … 76.21%
 Time  (median):     861.542 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   885.639 μs ± 121.070 μs  ┊ GC (mean ± σ):  2.43% ±  7.32%

  ▆▇██▇▅▄▃▂▁                                                    ▂
  ██████████▇▅▅▅▄▄▅▃▄▁▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇▇███▇▆▆▆▄▅▅▅▅▅▄▆▅▅▄ █
  828 μs        Histogram: log(frequency) by time       1.47 ms <

 Memory estimate: 1.79 MiB, allocs estimate: 19603.

In [6]:
@benchmark MaxPool_v2(input, (2,2), gradient)

BenchmarkTools.Trial: 6428 samples with 1 evaluation.
 Range (min … max):  718.125 μs …   3.292 ms  ┊ GC (min … max): 0.00% … 75.24%
 Time  (median):     763.375 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   776.933 μs ± 147.962 μs  ┊ GC (mean ± σ):  1.53% ±  5.67%

                       ▁▁▄▅█▇▇█▅▅▃▂▁                             
  ▂▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▅▆█████████████▇▇▆▆▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▄
  718 μs           Histogram: frequency by time          815 μs <

 Memory estimate: 306.33 KiB, allocs estimate: 3.

In [7]:
MaxPool_v1(input, (2,2), gradient) == MaxPool_v2(input, (2,2), gradient)

true